In [3]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client as FDSN_Client
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import os
from progress.bar import Bar
from tqdm.notebook import tqdm
from multiprocessing import Pool
from glob import glob

In [12]:
#load list of quakes
quakelist=pd.read_csv('data/quakelist1.txt',sep="|") 
quakelist.columns = quakelist.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

#load stationlist
stationlist=pd.read_csv('data/station_h_data.txt',sep="|")
stationlist.columns = stationlist.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [5]:
np.random.seed(42)
rand_inds=np.array(range(len(quakelist)))
np.random.shuffle(rand_inds)

In [6]:
client = FDSN_Client("GEONET")
client_nrt = FDSN_Client("https://service-nrt.geonet.org.nz")

In [13]:
import warnings
warnings.filterwarnings('ignore') # turning off warnings because otherwise obspy spams warnings

In [8]:
def close_to_quake(tt):
    #returns True if the time is within an hour (either side) of a quake in quakelist, otherwise False
    quaketime_diffs=[UTCDateTime(quaketime)-UTCDateTime(tt) for quaketime in quakelist['time'].to_numpy()]
    if any(np.abs(quaketime_diffs)<3600): #if any quake happens within an hour either side of the time
        too_close=True
    else:
        too_close=False
    return too_close

In [9]:
def good_stream(st1):
    #Function for determining whether a stream st1 is good data
    #Is it of at least 3 traces long
    #Does it have the first 3 traces channels as "HHZ", "HHE" or "HHN"
    #it have the correct number of points, 32000 or 32001
    #test for any std==0 
    stream_good=1
    if len(st1)<3: #is it at least 3 long
        return False
    else:
        filled=[0,0,0]
        for tr_ind in range(3): #testing that it has the 3 channels
            if st1[tr_ind].stats.channel=="HHZ":
                fill_ind=0
            elif st1[tr_ind].stats.channel=="HHN":
                fill_ind=1
            elif st1[tr_ind].stats.channel=="HHE":
                fill_ind=2
            else: 
                return False
            
            filled[fill_ind]=1
            if not (st1[tr_ind].stats.npts==32000 or st1[tr_ind].stats.npts==32001): #testing it has the correct npts
#             if not (st1[tr_ind].stats.npts==240001 or st1[tr_ind].stats.npts==240000):

                return False

            if np.std(st1[tr_ind].data)==0: #making sure its actually changing in time
                return False
    
        if not filled ==[1,1,1]: #making sure it had all the channels
            return False
        
    return True

In [10]:
def find_rand_times(n_samples,wait_time=3600):
    #generates n_sample random times (floats)
    #each of these times is not closer than wait_time (default 1 hour) on 
    #either side of one of the quakes in quakelist
    quaketimes=np.array(([float(UTCDateTime(quaketime)) for quaketime in quakelist['time'].to_numpy()]))
    max_time=max(quaketimes)
    min_time=min(quaketimes)
    rand_times=np.zeros(n_samples)
    ii=0
    
    while ii<n_samples:
        t_temp=np.random.uniform(min_time,max_time)
        if all(np.abs(quaketimes-t_temp)>wait_time):
            rand_times[ii]=t_temp
            ii+=1
#             if ii%20==0:
#                 print(ii)

    return rand_times

In [33]:
def quake_savefail_write(fail_id,data_dir,reason=''):
    savefail=open(data_dir+'quake_savefail.txt','a')
    savefail.write(fail_id+' '+ reason)
    savefail.write("\n")
    savefail.close()

In [38]:
data_dir='/media/peter/data/earthquakenz/data/test/' # data to save quakebloks to
station_list1=['APZ', 'BFZ', 'BKZ', 'CTZ',  'DSZ', 'EAZ', 'FOZ',
       'FWVZ', 'GLKZ', 'GRZ', 'GVZ', 'HAZ', 'HIZ', 'INZ', 'JCZ', 'KHEZ',
       'KHZ', 'KNZ',  'LBZ', 'LTZ', 'MLZ', 'MQZ',
       'MRZ', 'MSZ', 'MWZ', 'MXZ', 'NNZ', 'ODZ', 'OPRZ', 'OPZ', 'OTVZ',
       'OUZ', 'OXZ', 'PUZ', 'PXZ', 'PYZ', 'QRZ', 'RATZ', 'RIZ', 
       'RTZ', 'SYZ', 'THZ', 'TLZ', 'TMVZ', 'TOZ', 'TRVZ', 'TSZ', 'TUZ',
        'VRZ', 'WAZ', 'WCZ', 'WEL', 'WHVZ', 'WHZ', 'WIZ', 'WKZ',
       'WSRZ', 'WVZ'] #list of seismograph stations to use
#this list of stations took a long time to arrive at, a lot of the stations have bad data and so if the 
#station list was too long then there would be not many quakes/other time sections where they all work
#but you do want to have a substantial portion of them working, I think this is a good middle ground.

# rand_times=np.load(data_dir+'rand_times.npy') #loaded rand_times


def parallel_download_noquake(t_start,step=10):
    #This function takes a time t_start, and attempts to download a 5 minute time section
    #(10 second padding on either side) from all the stations 
    #in station_list1, from the HHZ,HHN, and HHE sensors. If the data from each seismograph is there and good
    #it adds the stream data to a blok. The step parameter tells it how many steps to take when saving 
    #the waveform in time to the blok. It then saves the blok to a .npy file.
    #If anything bad happens, like the download fails, the quake happened before the construction of a station,
    #the stream data is bad (for any stream), then the function stops and doesn't save a blok
    
    #also it has a cool progress bar for each blok, and can be run in parallel
    
    
    t_duration=5*60 #duration of the time section (without 10 second padding)
    samplerate=100 #the assumed sample rate of the traces
    blok=np.zeros((int(np.ceil(t_duration*samplerate/step)),3,len(station_list1))) #empty blok to save
    starttime=10 
    start_times=np.zeros((3,len(station_list1)))
    timeid=str(t_start) #t_start as a string, to be used as a label
#     print('Starting: '+ str(UTCDateTime(timeid)))
    print('Starting: '+ str((timeid)))
    for station_ind in tqdm(range(len(station_list1)),leave=False,desc=timeid): #for each station (with progressbar)
        #check that the station existed at the time
        if stationlist[stationlist['station']==station_list1[station_ind]]['starttime'].to_numpy()<UTCDateTime(t_start-10):
            attempts=0
            downloaded_stream=0
            while attempts <4: #4 attempts to download
                try:
    #                 st= client.get_waveforms("NZ",stationlist['station'][station_ind],"*", "HHZ,HHN,HHE,HH1,HH2", t-t_before, t+t_after ,minimumlength=t_before,attach_response=True)
                    # st is the stream which is downloaded, for the station
                    warnings.filterwarnings('ignore')
                    st= client.get_waveforms("NZ",station_list1[station_ind] ,"*", "HH?", UTCDateTime(t_start-10), UTCDateTime(t_start+5*60+10),attach_response=True)
                    warnings.filterwarnings('default')
                    downloaded_stream=1 #successfully downloaded!
#                   
                    break
                except:
                    attempts+=1 #incriment attempts if it failed
            
            if downloaded_stream: 
                if good_stream(st): #test stream is good
                    #also test that there are the correct channels for the traces
                    filled=[0,0,0]
                    for tr_ind in range(3):
                        if st[tr_ind].stats.channel=="HHZ":
                            fill_ind=0
                        elif st[tr_ind].stats.channel=="HHN":
                            fill_ind=1
                        elif st[tr_ind].stats.channel=="HHE":
                            fill_ind=2
                        else:
                            print('SOMETHING WRONG WITH THE HH* INDS')
                        #add to the blok! (if everything is good)
                        blok[:,fill_ind,station_ind]=st[tr_ind].data[starttime*samplerate:int(t_duration*samplerate)+starttime*samplerate:step]
                        filled[fill_ind]=1
                        start_times[fill_ind,station_ind]=float(st[tr_ind].stats.starttime) #start time of the trace

                    if not filled== [1,1,1]: #this tells us if we dont have a HHZ, HHN, HHE stream
                        print('NOT FILLED!')
                        return
                        
                    if station_ind>0: #this makes sure that the start times are close enough together
                        #its alright if they're about 1 sample out, but more than that and it might be bad
                        if np.max(np.abs(start_times[:,station_ind]-start_times[:,station_ind-1]))>1.5/samplerate:
                            print('Something off with start times around at ' + timeid+str(station_ind)+', '+station_list1[station_ind])
                            print(start_times[:,station_ind]-start_times[:,station_ind-1])
                            print(np.max(np.abs(start_times[:,station_ind]-start_times[:,station_ind-1])))
                else:
                    return
            else: 
                return
    save_attempts=0
    while save_attempts<5: #5 attempts to save this blok
        try:  
            np.save(data_dir+'noquakes/bloks1/'+timeid+'.npy',blok) #save blok to .npy file
            print('Ending: '+ str((timeid)))

            break
        except:
            save_attempts+=1 #incriminent attempts if it failed

In [41]:
for ii in range(10,14):
    parallel_download_noquake(rand_times[ii])

Starting: 1480296830.13


Starting: 1583858017.55


Starting: 1538155240.77


Starting: 1585274280.62


Ending: 1585274280.62


In [74]:
data_dir='/media/peter/data/earthquakenz/data/test/' # data to save quakebloks to
station_list1=['APZ', 'BFZ', 'BKZ', 'CTZ',  'DSZ', 'EAZ', 'FOZ',
       'FWVZ', 'GLKZ', 'GRZ', 'GVZ', 'HAZ', 'HIZ', 'INZ', 'JCZ', 'KHEZ',
       'KHZ', 'KNZ',  'LBZ', 'LTZ', 'MLZ', 'MQZ',
       'MRZ', 'MSZ', 'MWZ', 'MXZ', 'NNZ', 'ODZ', 'OPRZ', 'OPZ', 'OTVZ',
       'OUZ', 'OXZ', 'PUZ', 'PXZ', 'PYZ', 'QRZ', 'RATZ', 'RIZ', 
       'RTZ', 'SYZ', 'THZ', 'TLZ', 'TMVZ', 'TOZ', 'TRVZ', 'TSZ', 'TUZ',
        'VRZ', 'WAZ', 'WCZ', 'WEL', 'WHVZ', 'WHZ', 'WIZ', 'WKZ',
       'WSRZ', 'WVZ'] #list of seismograph stations to use
#this list of stations took a long time to arrive at, a lot of the stations have bad data and so if the 
#station list was too long then there would be not many quakes/other time sections where they all work
#but you do want to have a substantial portion of them working, I think this is a good middle ground.

t_before_rand=(15+np.random.rand(len(quakelist))*6-3)*60 #for each quake we take a section starting 
#between 12 and 18 mins before the quaketime 

def parallel_download_quake(quake_ind,step=10,save_fail=True):
    #This function takes a time t_start, and attempts to download a 5 minute time section
    #(10 second padding on either side) from all the stations 
    #in station_list1, from the HHZ,HHN, and HHE sensors. If the data from each seismograph is there and good
    #it adds the stream data to a blok. The step parameter tells it how many steps to take when saving 
    #the waveform in time to the blok. It then saves the blok to a .npy file.
    #If anything bad happens, like the download fails, the quake happened before the construction of a station,
    #the stream data is bad (for any stream), then the function stops and doesn't save a blok
    save_dir=data_dir+'quakes/bloks1/'
    #also it has a cool progress bar for each blok, and can be run in parallel
    quakeid=quakelist['eventid'][quake_ind]
    t_start=float(UTCDateTime(quakelist['time'][quake_ind]))-t_before_rand[quake_ind] #time that the
    t_duration=5*60 #duration of the time section (without 10 second padding)
    samplerate=100 #the assumed sample rate of the traces
    blok=np.zeros((int(np.ceil(t_duration*samplerate/step)),3,len(station_list1))) #empty blok to save
    starttime=10 
    start_times=np.zeros((3,len(station_list1)))
    timeid=str(t_start) #t_start as a string, to be used as a label
    print('Starting: '+ str((quakeid)))
    for station_ind in tqdm(range(len(station_list1)),leave=False,desc=quakeid): #for each station (with progressbar)
        #check that the station existed at the time
        if stationlist[stationlist['station']==station_list1[station_ind]]['starttime'].to_numpy()<UTCDateTime(t_start-10):
            attempts=0
            downloaded_stream=0
            while attempts <4: #4 attempts to download
                try:
    #                 st= client.get_waveforms("NZ",stationlist['station'][station_ind],"*", "HHZ,HHN,HHE,HH1,HH2", t-t_before, t+t_after ,minimumlength=t_before,attach_response=True)
                    # st is the stream which is downloaded, for the station
                    warnings.filterwarnings('ignore')
                    st= client.get_waveforms("NZ",station_list1[station_ind] ,"*", "HH?", UTCDateTime(t_start-10), UTCDateTime(t_start+5*60+10),attach_response=True)
                    warnings.filterwarnings('default')
                    downloaded_stream=1 #successfully downloaded!
#                   
                    break
                except:
                    attempts+=1 #incriment attempts if it failed
            
            if downloaded_stream: 
                if good_stream(st): #test stream is good
                    #also test that there are the correct channels for the traces
                    filled=[0,0,0]
                    for tr_ind in range(3):
                        if st[tr_ind].stats.channel=="HHZ":
                            fill_ind=0
                        elif st[tr_ind].stats.channel=="HHN":
                            fill_ind=1
                        elif st[tr_ind].stats.channel=="HHE":
                            fill_ind=2
                        else:
                            print('SOMETHING WRONG WITH THE HH* INDS')
                        #add to the blok! (if everything is good)
                        blok[:,fill_ind,station_ind]=st[tr_ind].data[starttime*samplerate:int(t_duration*samplerate)+starttime*samplerate:step]
                        filled[fill_ind]=1
                        start_times[fill_ind,station_ind]=float(st[tr_ind].stats.starttime) #start time of the trace

                    if not filled== [1,1,1]: #this tells us if we dont have a HHZ, HHN, HHE stream
                        print('NOT FILLED!')
                        quake_savefail_write(quakeid+' '+station_list1[station_ind],save_dir,'notfilled')
                        return
                        
                    if station_ind>0: #this makes sure that the start times are close enough together
                        #its alright if they're about 1 sample out, but more than that and it might be bad
                        if np.max(np.abs(start_times[:,station_ind]-start_times[:,station_ind-1]))>1.5/samplerate:
                            print('Something off with start times around at ' + timeid+str(station_ind)+', '+station_list1[station_ind])
                            print(start_times[:,station_ind]-start_times[:,station_ind-1])
                            print(np.max(np.abs(start_times[:,station_ind]-start_times[:,station_ind-1])))
                else:
                    quake_savefail_write(quakeid+' '+station_list1[station_ind],save_dir,'notgoodstream')
                    return
            else:
                quake_savefail_write(quakeid+' '+station_list1[station_ind],save_dir,'notdownloaded')
                return
        else:
            return
    save_attempts=0
    while save_attempts<5: #5 attempts to save this blok
        try:  
            np.save(save_dir+quakeid+'.npy',blok) #save blok to .npy file
            print('Ending: '+ str((quakeid)))

            break
        except:
            save_attempts+=1 #incriminent attempts if it failed
    if save_attempts==5:
        quake_savefail_write(quakeid+' '+station_list1[station_ind],save_dir,'savingfailed')

In [75]:
for ii in range(4,10):
    parallel_download_quake(ii)

Starting: 2016p934797


Starting: 2016p868025


Ending: 2016p868025
Starting: 2016p935569


Starting: 2017p066354


Starting: 2017p071831


Starting: 2016p867831


Ending: 2016p867831


In [56]:
def normalise_bloks(blok_dir,bloknorm_dir):
    #normalised all the bloks in blok_dir, and saves in the new directory bloknorm_dir
    file_list=os.listdir(blok_dir)#list all the files in the blok dir
    
    for filename in file_list: #for each block
        if filename[-4:]=='.npy': #make sure its actually a data file, not a txt or something else
            blok=np.load(blok_dir+filename) #load unnormalised blok
            blok_norm=np.zeros(np.shape(blok))
            blok_std=np.std(blok,axis=0) #std for each channel
            blok_mean=np.mean(blok,axis=0) #mean for each channel
            for ii in range(3): #for each channel
                for station_ind in range(blok.shape[2]): #for each station
                    #normalise the blok
                    blok_norm[:,ii,station_ind]=(blok[:,ii,station_ind]-blok_mean[ii,station_ind])/blok_std[ii,station_ind]
            np.save(bloknorm_dir+filename,blok_norm) #save the normalised blok in bloknorm_dir